<a href="https://colab.research.google.com/github/Andre-1970/Machine_learning_classical_algorithms_Sem4/blob/main/3_11_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Домашняя работа

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома, где будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.datasets import fetch_openml

In [2]:
degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
best_score = 0.0
best_degree = 0

In [3]:
df = pd.read_csv('data/phones.csv')
# разделение признаков и целевой переменной
X = df[['disk', 'year']]
y = df['price']

In [4]:
# Переберем все степени и найдем лучший R2-score
for degree in degrees:
    # Создадим полиномиальные признаки
    poly_features = PolynomialFeatures(degree)
    X_poly = poly_features.fit_transform(X)
    # Обучим модель
    model = LinearRegression()
    model.fit(X_poly, y)

    # Вычислим R2-score
    score = r2_score(y, model.predict(X_poly))

    # Если R2-score лучше, чем текущий лучший, обновим переменные
    if score > best_score:
        best_score = score
        best_degree = degree

In [5]:
# Выведем лучшую степень и соответствующий ей R2-score
print("Best degree:", best_degree)
print("Best R2-score:", best_score)

# Обучим лучшую модель и построим предсказания
poly_features = PolynomialFeatures(best_degree)
X_poly = poly_features.fit_transform(X)
model = LinearRegression()
model.fit(X_poly, y)
y_pred = model.predict(X_poly)

# Выведем предсказания
print("Predictions:", y_pred)

Best degree: 9
Best R2-score: 0.9943138168888681
Predictions: [ 7562.57611915  7345.33051655  1677.40295002 14835.58109257
  4043.55809948 15810.70185539  7345.33051655  4658.04765007
 12707.54276535  6582.52661106 13790.47771332 10759.59111091
  9557.69453308  5214.90372536  7008.15501282  2250.62603542
 12707.54276535  5998.16981384 13158.91097996 12090.86844704
  1383.44454643  4658.04765007 10558.26303837  9620.38674137
  8571.01982281  6582.52661106  8441.97591278  9038.25140259
  2380.7844331  12707.54276535 11619.22429058  4870.86700889
  7008.15501282  4994.32917377  7562.57611915  5958.78547356
 12707.54276535 15810.70185539  7562.57611915  4658.04765007
 14835.58109257 12707.54276535  5257.93419525  8441.97591278
  7562.57611915  7345.33051655  9620.38674137  2699.55728886
 10558.26303837 13158.91097996 10042.74772903  8030.69995758
  3133.17899963  1677.40295002  8030.69995758  7871.35931179
  1431.85556766 12707.54276535 10558.26303837 12090.86844704
 10558.26303837  4658.0

**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисляемых коэффициентов. 

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [6]:
class CustomLinearReg:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        self.coef_ = np.linalg.inv(X.T @ X) @ X.T @ y
        self.intercept_ = self.coef_[0]
        self.coef_ = self.coef_[1:]

    def predict(self, X):
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return X @ np.hstack((self.intercept_, self.coef_))

In [7]:
df = pd.read_csv('data/phones.csv')
# разделение признаков и целевой переменной
X = df[['disk', 'year']]
y = df['price']

# разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = CustomLinearReg()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}')

MAE: 370.5450395136257
MSE: 207228.6017281483
RMSE: 455.2236831802013


**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid) с помощью функции `train_test_split` из библиотеки `sklearn`
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [61]:
boston_dataset = fetch_openml(name='boston')
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
print(boston.head())

      CRIM    ZN  INDUS CHAS    NOX     RM   AGE     DIS RAD    TAX  PTRATIO  \
0  0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296.0     15.3   
1  0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242.0     17.8   
2  0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242.0     17.8   
3  0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222.0     18.7   
4  0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222.0     18.7   

        B  LSTAT  
0  396.90   4.98  
1  396.90   9.14  
2  392.83   4.03  
3  394.63   2.94  
4  396.90   5.33  


C:\___project\python\Machine_learning_classical_algorithms_Sem4\venv\lib\site-packages\sklearn\datasets\_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
C:\___project\python\Machine_learning_classical_algorithms_Sem4\venv\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [79]:
non_numeric_columns = boston.select_dtypes(include=['category']).columns.tolist()
for name in non_numeric_columns:
    boston[name] = boston[name].astype('float64')

boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [88]:
#1
X = boston.drop('target', axis=1)
y = boston.target

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

#2
model = LinearRegression()
model.fit(X_train, y_train)

#3
y_pred = model.predict(X_valid)
print(y_pred)

#4
r2 = r2_score(y_valid, y_pred)
print(f'R2 score: {r2}')

[28.99672362 36.02556534 14.81694405 25.03197915 18.76987992 23.25442929
 17.66253818 14.34119    23.01320703 20.63245597 24.90850512 18.63883645
 -6.08842184 21.75834668 19.23922576 26.19319733 20.64773313  5.79472718
 40.50033966 17.61289074 27.24909479 30.06625441 11.34179277 24.16077616
 17.86058499 15.83609765 22.78148106 14.57704449 22.43626052 19.19631835
 22.43383455 25.21979081 25.93909562 17.70162434 16.76911711 16.95125411
 31.23340153 20.13246729 23.76579011 24.6322925  13.94204955 32.25576301
 42.67251161 17.32745046 27.27618614 16.99310991 14.07009109 25.90341861
 20.29485982 29.95339638 21.28860173 34.34451856 16.04739105 26.22562412
 39.53939798 22.57950697 18.84531367 32.72531661 25.0673037  12.88628956
 22.68221908 30.48287757 31.52626806 15.90148607 20.22094826 16.71089812
 20.52384893 25.96356264 30.61607978 11.59783023 20.51232627 27.48111878
 11.01962332 15.68096344 23.79316251  6.19929359 21.6039073  41.41377225
 18.76548695  8.87931901 20.83076916 13.25620627 20

In [89]:
#применение z-преобразования
z_X_train = (X_train - X_train.mean()) / X_train.std()
z_X_valid = (X_valid - X_valid.mean()) / X_valid.std()

#2
model = LinearRegression()
model.fit(z_X_train, y_train)

#3
z_y_pred = model.predict(z_X_valid)
print(z_y_pred)

#4
z_r2 = r2_score(y_valid, z_y_pred)
print(f'R2 score: {z_r2}')

print(f'Изменение R2 после применения z-преобразования: {z_r2 - r2}')

[31.32141926 38.34541052 16.58140478 26.57522306 19.93035763 25.16444263
 19.385686   15.78559222 24.33265774 22.27734901 27.39702904 20.85314996
 -6.10221485 23.5725713  20.55562866 27.6411157  22.29649     6.53674576
 43.44143521 18.66845004 28.92183863 32.07132489 12.44729253 25.56675924
 18.90412441 16.8529762  24.49761815 15.72936632 24.45628755 20.73101348
 24.55864222 26.72707585 27.21648499 18.48151062 18.05719421 19.13606178
 33.35435437 21.22882794 25.75442313 26.24521667 15.42797704 34.18994554
 45.61371152 18.77748368 29.35733019 17.99300145 15.17010984 27.5642953
 21.55636132 32.44683349 23.50187845 36.95046121 17.27141279 27.74147087
 42.73910533 23.75115832 19.9587603  34.3745656  26.87261094 14.12238181
 23.96748336 32.20281507 33.55284288 16.85049    21.64481952 18.59440578
 21.74500974 27.56057587 32.3639105  12.61941854 22.26773345 28.85183546
 11.92572786 16.03364553 25.63780968  6.62238647 23.01682534 44.30872591
 19.94330116 10.5279837  22.32848502 14.48442071 22.